In [1]:
# import numpy as np
import numpy.linalg as la
import autograd.numpy as np
from autograd import grad, jacobian
from scipy.optimize import line_search, fsolve, linprog
from fractions import Fraction

# Class test 1


## Matrix definiteness

- positive (semi-positive) definite : all eigenvalue positive (non-negative)
- negative (semi-negative) definite : all eigenvalue negative (non-possible)
- indefinite


In [26]:
def positive_definite(A):
    """
    Check if the matrix A is positive definite.
    """
    return np.all(la.eigvals(A) > 0)


def negative_definite(A):
    """
    Check if the matrix A is positive definite.
    """
    return np.all(la.eigvals(A) < 0)


def semi_positive_definite(A):
    """
    Check if the matrix A is positive definite.
    """
    return np.all(la.eigvals(A) >= 0)


def semi_negative_definite(A):
    """
    Check if the matrix A is positive definite.
    """
    return np.all(la.eigvals(A) <= 0)


A = np.array([[3, 1, 5], [1, 4, 2], [5, 2, 1]])


positive_definite(A)  # Check if A is positive definite

np.False_

## Matrix symetry


In [27]:
def symetry(A: np.ndarray):
    """
    Check if the matrix A is positive definite.
    """
    if len(A.shape) != 2:
        raise False
    if A.shape[0] != A.shape[1]:
        raise False
    return np.allclose(A, A.T)


A = np.array([[3, 1, 5], [1, 4, 2], [5, 2, 1]])
symetry(A)  # Check if A is positive definite

True

## Matrix inverse

A square matrix is invertible if

- determinant is not zero
- SVD?
- Eigenvalue?


In [28]:
la.det(A)

np.float64(-80.99999999999996)

## Auto-differentiation


In [29]:
def f(x):
    return np.sum(x**2) + x[0] * x[1] - x[0] - 2 * x[1]


grad_f = grad(f)
hessen_f = jacobian(grad_f)
x = np.array([1.0, -1.0])
print(
    " gradient = ",
    grad_f(x),
    "\n Hessian = ",
    hessen_f(x),
    "\n determinant = ",
    la.det(hessen_f(x)),
)

 gradient =  [ 0. -3.] 
 Hessian =  [[2. 1.]
 [1. 2.]] 
 determinant =  2.9999999999999996


## Steepest Descent Method


In [30]:
def steepest_descent(
    fun, x0: np.ndarray, jac, ls=line_search, maxiter=100, amax=1000.0, tol=1.0e-8
):
    x_eps = tol  # tolerence for convergence on delta x
    f_eps = tol  # tolerence for convergence on delta f
    g_eps = tol  # tolerence for convergence on norm of gradient
    x_k = x0.copy()
    nit = 1
    f_k = fun(x_k)
    d_k = -jac(x_k)
    if la.norm(jac(x_k)) < g_eps:
        print("norm of gradient is within tolerence")
        return None
    while True:
        print("Interation: ", nit)
        print(f"x_{nit} = ", x_k)
        print(f"f_{nit} = ", f_k)
        print(f"g_{nit} = ", jac(x_k))
        print(f"d_{nit} = ", d_k)
        alpha_k, _, _, _, _, success = ls(fun, jac, x_k, d_k, amax=amax)
        if success is None:
            print("Line search fail")
            break

        print("alpha_k = ", alpha_k)
        if abs(alpha_k * la.norm(jac(x_k))) < x_eps:
            print("change of x is within tolerence")
            break

        x_k1 = x_k + alpha_k * d_k
        d_k1 = -jac(x_k1)

        if abs(f_k - fun(x_k1)) < f_eps:
            print("change of fun is within tolerence")
            break
        if la.norm(jac(x_k1)) < g_eps:
            print("norm of gradient is within tolerence")
            break

        if nit > maxiter:
            print("Max iter reached")
            break

        nit += 1
        x_k = x_k1
        f_k = fun(x_k1)
        d_k = d_k1

## Fletcher-Reeves Method


In [31]:
def fletcher_reeves(
    fun, x0: np.ndarray, jac, ls=line_search, maxiter=100, amax=1000.0, tol=1.0e-8
):
    x_eps = tol  # tolerence for convergence on delta x
    f_eps = tol  # tolerence for convergence on delta f
    g_eps = tol  # tolerence for convergence on norm of gradient
    x_k = x0.copy()
    nit = 1
    f_k = fun(x_k)
    d_k = -jac(x_k)
    if la.norm(jac(x_k)) < g_eps:
        print("norm of gradient is within tolerence")
        return None
    while True:
        print("Interation: ", nit)
        print(f"x_{nit} = ", x_k)
        print(f"f_{nit} = ", f_k)
        print(f"g_{nit} = ", jac(x_k))
        print(f"d_{nit} = ", d_k)
        alpha_k, _, _, _, _, success = ls(fun, jac, x_k, d_k, amax=amax)
        if success is None:
            print("Line search fail")
            break

        print("alpha_k = ", alpha_k)
        if abs(alpha_k * la.norm(jac(x_k))) < x_eps:
            print("change of x is within tolerence")
            break

        x_k1 = x_k + alpha_k * d_k
        d_k1 = -jac(x_k1) + (la.norm(jac(x_k1)) ** 2 / la.norm(jac(x_k)) ** 2) * d_k

        if abs(f_k - fun(x_k1)) < f_eps:
            print("change of fun is within tolerence")
            break
        if la.norm(jac(x_k1)) < g_eps:
            print("norm of gradient is within tolerence")
            break

        if nit > maxiter:
            print("Max iter reached")
            break

        nit += 1
        x_k = x_k1
        f_k = fun(x_k1)
        d_k = d_k1

## Fletcher-Reeves Reset Method


In [32]:
def fletcher_reeves_reset(
    fun,
    x0: np.ndarray,
    jac,
    ls=line_search,
    maxiter=100,
    amax=1000.0,
    tol=1.0e-8,
    reset=100,
):
    x_eps = tol  # tolerence for convergence on delta x
    f_eps = tol  # tolerence for convergence on delta f
    g_eps = tol  # tolerence for convergence on norm of gradient
    x_k = x0.copy()
    nit = 1
    f_k = fun(x_k)
    d_k = -jac(x_k)
    if la.norm(jac(x_k)) < g_eps:
        print("norm of gradient is within tolerence")
        return None
    while True:
        print("Interation:", nit)
        print(f"x_{nit} = ", x_k)
        print(f"f_{nit} = ", f_k)
        print(f"g_{nit} = ", jac(x_k))
        print(f"d_{nit} = ", d_k)
        alpha_k, _, _, _, _, success = ls(fun, jac, x_k, d_k, amax=amax)
        if success is None:
            print("Line search fail")
            break

        print("alpha_k = ", alpha_k)
        if abs(alpha_k * la.norm(jac(x_k))) < x_eps:
            print("change of x is within tolerence")
            break
        x_k1 = x_k + alpha_k * d_k
        d_k1 = None
        if nit % reset == 0:
            d_k1 = -jac(x_k1)
        else:
            d_k1 = -jac(x_k1) + (la.norm(jac(x_k1)) ** 2 / la.norm(jac(x_k)) ** 2) * d_k

        if abs(f_k - fun(x_k1)) < f_eps:
            print("change of fun is within tolerence")
            break
        if la.norm(jac(x_k1)) < g_eps:
            print("norm of gradient is within tolerence")
            break

        if nit > maxiter:
            print("Max iter reached")
            break
        nit += 1
        x_k = x_k1
        f_k = fun(x_k1)
        d_k = d_k1

## Newton Method


In [33]:
def newton(
    fun,
    x0: np.ndarray,
    jac,
    ls=line_search,
    maxiter=100,
    amax=1000.0,
    tol=1.0e-8,
):
    x_eps = tol  # tolerence for convergence on delta x
    f_eps = tol  # tolerence for convergence on delta f
    g_eps = tol  # tolerence for convergence on norm of gradient
    hessian = jacobian(jac)
    x_k = x0.copy()
    nit = 1
    f_k = fun(x_k)
    d_k = -la.inv(hessian(x_k)) @ jac(x_k)
    if la.norm(jac(x_k)) < g_eps:
        print("norm of gradient is within tolerence")
        return None
    while True:
        print("Interation:", nit)
        print(f"x_{nit} = ", x_k)
        print(f"f_{nit} = ", f_k)
        print(f"g_{nit} = ", jac(x_k))
        print(f"d_{nit} = ", d_k)
        x_k1 = x_k + d_k
        d_k1 = -la.inv(hessian(x_k)) @ jac(x_k1)
        if abs(f_k - fun(x_k1)) < f_eps:
            print("change of fun is within tolerence")
            break
        if la.norm(jac(x_k1)) < g_eps:
            print("norm of gradient is within tolerence")
            break

        if nit > maxiter:
            print("Max iter reached")
            break
        nit += 1
        x_k = x_k1
        f_k = fun(x_k1)
        d_k = d_k1

## Modified Newton Method


In [34]:
def newton_modified(
    fun,
    x0: np.ndarray,
    jac,
    ls=line_search,
    maxiter=100,
    amax=1000.0,
    tol=1.0e-8,
):
    x_eps = tol  # tolerence for convergence on delta x
    f_eps = tol  # tolerence for convergence on delta f
    g_eps = tol  # tolerence for convergence on norm of gradient
    hessian = jacobian(jac)
    x_k = x0.copy()
    nit = 1
    f_k = fun(x_k)
    d_k = -la.inv(hessian(x_k)) @ jac(x_k)
    if la.norm(jac(x_k)) < g_eps:
        print("norm of gradient is within tolerence")
        return None
    while True:
        print("Interation:", nit)
        print(f"x_{nit} = ", x_k)
        print(f"f_{nit} = ", f_k)
        print(f"g_{nit} = ", jac(x_k))
        print(f"d_{nit} = ", d_k)
        alpha_k, _, _, _, _, success = ls(fun, jac, x_k, d_k, amax=amax)
        if success is None:
            print("Line search fail")
            break

        print("alpha_k = ", alpha_k)
        if abs(alpha_k * la.norm(d_k)) < x_eps:
            print("change of x is within tolerence")
            break
        x_k1 = x_k + alpha_k * d_k
        d_k1 = -la.inv(hessian(x_k)) @ jac(x_k1)
        if abs(f_k - fun(x_k1)) < f_eps:
            print("change of fun is within tolerence")
            break
        if la.norm(jac(x_k1)) < g_eps:
            print("norm of gradient is within tolerence")
            break

        if nit > maxiter:
            print("Max iter reached")
            break
        nit += 1
        x_k = x_k1
        f_k = fun(x_k1)
        d_k = d_k1

## Davidon-Fletcher-Powell Method


In [50]:
def davidon_fletcher_powell(
    fun,
    x0: np.ndarray,
    jac,
    H1: np.ndarray,
    ls=line_search,
    maxiter=100,
    amax=1000.0,
    tol=1.0e-8,
):
    if not symetry(H1) or not positive_definite(H1):
        raise ValueError("H1 must be a symetric positive definite matrix")
    x_eps = tol  # tolerence for convergence on delta x
    f_eps = tol  # tolerence for convergence on delta f
    g_eps = tol  # tolerence for convergence on norm of gradient
    hessian = jacobian(jac)
    x_k = x0.copy()
    nit = 1
    f_k = fun(x_k)
    H_k = H1.copy()
    d_k = -H_k @ jac(x_k)
    if la.norm(jac(x_k)) < g_eps:
        print("norm of gradient is within tolerence")
        return None
    while True:
        print("Interation:", nit)
        print(f"x_{nit} = ", x_k)
        print(f"f_{nit} = ", f_k)
        print(f"g_{nit} = ", jac(x_k))
        print(f"d_{nit} = ", d_k)
        print(f"H_{nit} = ", H_k)
        alpha_k, _, _, _, _, success = ls(fun, jac, x_k, d_k, amax=amax)
        if nit == 2:
            alpha_k = 5 / 6
        if success is None:
            print("Line search fail")
            break

        print(f"alpha_{nit} = ", alpha_k)
        if abs(alpha_k * la.norm(d_k)) < x_eps:
            print("change of x is within tolerence")
            break
        x_k1 = x_k + alpha_k * d_k
        p_k = alpha_k * d_k
        q_k = jac(x_k1) - jac(x_k)
        H_k1 = (
            H_k
            + np.outer(p_k, p_k) / (p_k.T @ q_k)
            - np.outer(H_k @ q_k, H_k @ q_k) / (q_k.T @ H_k @ q_k)
        )
        d_k1 = -H_k1 @ jac(x_k1)
        if abs(f_k - fun(x_k1)) < f_eps:
            print("change of fun is within tolerence")
            break
        if la.norm(jac(x_k1)) < g_eps:
            print("norm of gradient is within tolerence")
            break

        if nit > maxiter:
            print("Max iter reached")
            break
        print(f"p_{nit}", p_k)
        print(f"q_{nit}", q_k)
        nit += 1
        x_k = x_k1
        f_k = fun(x_k1)
        d_k = d_k1
        H_k = H_k1

# Class test 2


In [2]:
def printer(x: np.ndarray):
    """
    Print the value of x
    """
    if len(x.shape) == 2:
        for i in range(len(x)):
            print(
                [f"{Fraction(x[i, j]).limit_denominator()}" for j in range(len(x[i]))]
            )
    else:
        print([f"{Fraction(x[i]).limit_denominator()}" for i in range(len(x))])

## Reduced Gradient


In [3]:
def basis_selection(arr, n):
    sorted_indices = np.argsort(arr)  # Ascending order
    largest_indices = sorted_indices[-n:][::-1]  # n largest, descending
    rest_indices = sorted_indices[:-n]  # The rest, still ascending
    largest_indices.sort()
    rest_indices.sort()
    return largest_indices, rest_indices


def reduce_gradient(
    fun, x0: np.ndarray, A: np.ndarray, b: np.ndarray, maxiter=100, tol=1.0e-8
):
    jac = grad(fun)
    m = A.shape[0]
    n = A.shape[1]
    nit = 1
    x = x0.copy()
    while True:
        print("______________________________")
        print("ITERATION:", nit)
        print("-----DIRECTION SEARCH-----")
        basis_index, non_index = basis_selection(x, m)
        print("basis_index =", basis_index + 1)
        xb = x[basis_index]
        xn = x[non_index]
        gra = jac(x)
        B = A[:, basis_index]
        print("B =")
        printer(B)
        print("B-1 =")
        printer(la.inv(B))
        N = A[:, non_index]
        print("N =")
        printer(N)
        gb = gra[basis_index]
        print("g =")
        printer(jac(x))
        print("gb =")
        printer(gb)
        gn = gra[non_index]
        print("gn =")
        printer(gn)
        rn = gn - gb.T @ la.inv(B) @ N
        print("rn =")
        printer(rn)
        dn = np.zeros(n - m)
        for i in range(n - m):
            if rn[i] <= 0:
                dn[i] = -rn[i]
            else:
                dn[i] = -xn[i] * rn[i]
        print("dn =")
        printer(dn)
        if np.all(dn == 0):
            print("Optimization terminated")
            break
        db = -la.inv(B) @ N @ dn
        print("db =")
        printer(db)
        dk = np.zeros(n)
        b_count = 0
        n_count = 0
        for i in range(n):
            if i in basis_index:
                dk[i] = db[b_count]
                b_count += 1
            else:
                dk[i] = dn[n_count]
                n_count += 1
        print("dk =")
        printer(dk)
        bounds = [0, np.inf]
        for i in range(n):
            if dk[i] > 0:
                bc = -x[i] / dk[i]
                if bc > bounds[0]:
                    bounds[0] = bc
            elif dk[i] < 0:
                bc = -x[i] / dk[i]
                if bc < bounds[1]:
                    bounds[1] = bc
        print("-----LINE SEARCH-----")
        print("bounds =")
        printer(np.array(bounds))
        ls = lambda alpha: fun(x + alpha * dk)
        gs = grad(ls)
        a = fsolve(gs, 1)[0]
        if a < bounds[0]:
            a = bounds[0]
        elif a > bounds[1]:
            a = bounds[1]
        print("alpha =", str(Fraction(a).limit_denominator()))
        x = x + a * dk
        print("x =")
        printer(x)
        nit += 1

## Gradient Projection


In [4]:
def gradient_projection(
    fun,
    x0: np.ndarray,
    A: np.ndarray,
    b: np.ndarray,
    W: np.ndarray = None,
    w: np.ndarray = None,
    maxiter=100,
    tol=1.0e-8,
):
    jac = grad(fun)
    nit = 1
    x = x0.copy()
    while True:
        if nit > maxiter:
            print("Max iter reached")
            break
        print("______________________________")
        print("ITERATION:", nit)
        print("-----DIRECTION SEARCH-----")
        print("grad =")
        printer(jac(x))
        active_index = np.where(np.abs(A @ x - b) <= tol)[0]
        print("active_index =")
        printer(active_index + 1)
        A_active = A[active_index, :]
        print("A_active =")
        printer(A_active)
        b_active = b[active_index]
        print("b_active =")
        printer(b_active)
        inactive_index = np.setdiff1d(np.arange(A.shape[0]), active_index)
        A_non_active = A[inactive_index, :]
        b_non_active = b[inactive_index]
        M = A_active
        dk = None
        if W is not None:
            M = np.vstack((A_active, W))
        print("M =")
        printer(M)
        # print("M @ M.T =")
        # printer(M @ M.T)
        # print("(M @ M.T)^-1 =")
        # printer(la.inv(M @ M.T))
        # print("((M @ M.T)^-1)@M =")
        # printer(la.inv(M @ M.T) @ M)
        # print("M.T @ ((M @ M.T)^-1) @ M =")
        # printer(M.T @ la.inv(M @ M.T) @ M)
        while True:
            if len(M) == 0:
                if np.all(np.abs(jac(x)) <= tol):
                    print("grad =", 0)
                    print("Optimization terminated due to gradient = 0")
                    return None
                else:
                    dk = -jac(x)
                    print("step 4 dk =")
                    printer(dk)
                    break
            else:
                dk = -(np.eye(M.shape[1]) - M.T @ la.inv(M @ M.T) @ M) @ jac(x)
                print("dk =")
                printer(dk)
                if np.all(np.abs(dk) <= tol):
                    dk = np.zeros(len(dk))
                    laragian = -la.inv(M @ M.T) @ M @ jac(x)
                    print("Lagrangian =")
                    printer(laragian)
                    muy = laragian[: len(A)]
                    # print("muy =")
                    # printer(muy)
                    if np.all(muy >= 0):
                        print("Optimization terminated due to muy >= 0")
                        return None
                    else:
                        remove_index = np.where(muy < 0)[0][0]
                        M = np.delete(M, remove_index, axis=0)
                        print("Modified matrix =")
                        printer(M)
                        # print("M @ M.T =")
                        # printer(M @ M.T)
                        # print("(M @ M.T)^-1 =")
                        # printer(la.inv(M @ M.T))
                        # print("((M @ M.T)^-1)@M =")
                        # printer(la.inv(M @ M.T) @ M)
                        # print("M.T @ ((M @ M.T)^-1) @ M =")
                        # printer(M.T @ la.inv(M @ M.T) @ M)
                else:
                    break
        print("-----LINE SEARCH-----")
        RHS = b_non_active - A_non_active @ x
        # print("RHS =")
        # printer(RHS)
        LHS = A_non_active @ dk
        # print("LHS =")
        # printer(LHS)
        bounds = [0, np.inf]
        for i in range(len(LHS)):
            if LHS[i] > 0:
                bc = RHS[i] / LHS[i]
                if bc < bounds[1]:
                    bounds[1] = bc
            elif LHS[i] < 0:
                bc = RHS[i] / LHS[i]
                if bc > bounds[0]:
                    bounds[0] = bc
        a_max = max(bounds[0], bounds[1], 0)
        # print("a_max =", str(Fraction(a_max).limit_denominator()))
        bounds = [0, a_max]
        print("bounds =")
        printer(np.array(bounds))
        ls = lambda alpha: fun(x + alpha * dk)
        gs = grad(ls)
        a = fsolve(gs, 1)[0]
        if a < bounds[0]:
            a = bounds[0]
        elif a > bounds[1]:
            a = bounds[1]
        print("alpha =", str(Fraction(a).limit_denominator()))
        x = x + a * dk
        print("x =")
        printer(x)
        nit += 1

## Method of Zoutendijk


In [17]:
def zoutendijk(
    fun,
    x0: np.ndarray,
    A: np.ndarray,
    b: np.ndarray,
    W: np.ndarray = None,
    w: np.ndarray = None,
    maxiter=100,
    tol=1.0e-8,
):
    jac = grad(fun)
    nit = 1
    x = x0.copy()
    n = len(x0)
    while True:
        if nit > maxiter:
            print("Max iter reached")
            break
        print("______________________________")
        print("ITERATION:", nit)
        print("-----DIRECTION SEARCH-----")
        gra = jac(x)
        print("grad =")
        printer(gra)
        active_index = np.where(np.abs(A @ x - b) <= tol)[0]
        print("active_index =")
        printer(active_index + 1)
        A_active = A[active_index, :]
        print("A_active =")
        printer(A_active)
        b_active = b[active_index]
        print("b_active =")
        printer(b_active)
        inactive_index = np.setdiff1d(np.arange(A.shape[0]), active_index)
        A_non_active = A[inactive_index, :]
        print("A_non_active =")
        printer(A_non_active)
        b_non_active = b[inactive_index]
        print("b_non_active =")
        printer(b_non_active)
        dk = np.array(
            linprog(
                gra,
                A_ub=A_active,
                b_ub=np.zeros(len(A_active)),
                bounds=[(-1, 1), (-1, 1)],
                method="highs",
            ).x
        )
        print("dk =")
        printer(dk)
        if np.abs(gra.T @ dk) <= tol:
            print("Optimization terminated due to grad * dk = 0")
            break
        print("-----LINE SEARCH-----")
        RHS = b_non_active - A_non_active @ x
        print("RHS =")
        printer(RHS)
        LHS = A_non_active @ dk
        print("LHS =")
        printer(LHS)
        bounds = [0, np.inf]
        for i in range(len(LHS)):
            if LHS[i] > 0:
                bc = RHS[i] / LHS[i]
                if bc < bounds[1]:
                    bounds[1] = bc
            elif LHS[i] < 0:
                bc = RHS[i] / LHS[i]
                if bc > bounds[0]:
                    bounds[0] = bc
        a_max = max(bounds[0], bounds[1], 0)
        print("a_max =", str(Fraction(a_max).limit_denominator()))
        bounds = [0, a_max]
        print("bounds =")
        printer(np.array(bounds))
        ls = lambda alpha: fun(x + alpha * dk)
        gs = grad(ls)
        a = fsolve(gs, 1)[0]
        if a < bounds[0]:
            a = bounds[0]
        elif a > bounds[1]:
            a = bounds[1]

        if nit == 2:
            a = 5
        print("alpha =", str(Fraction(a).limit_denominator()))
        x = x + a * dk
        print("x =")
        printer(x)
        nit += 1

## Test

In [18]:
f = lambda x: x[0] ** 2 - x[1] ** 2 + x[0] * x[1]
A = np.array([[-1, 3], [2, 1], [-1, -1]])
b = np.array([8, 4, 1])
x0 = np.array([1.0, 0.0])
zoutendijk(f, x0, A, b)

______________________________
ITERATION: 1
-----DIRECTION SEARCH-----
grad =
['2', '1']
active_index =
[]
A_active =
b_active =
[]
A_non_active =
['-1', '3']
['2', '1']
['-1', '-1']
b_non_active =
['8', '4', '1']
dk =
['-1', '-1']
-----LINE SEARCH-----
RHS =
['9', '2', '2']
LHS =
['-2', '-3', '2']
a_max = 1
bounds =
['0', '1']
alpha = 1
x =
['0', '-1']
______________________________
ITERATION: 2
-----DIRECTION SEARCH-----
grad =
['-1', '2']
active_index =
['3']
A_active =
['-1', '-1']
b_active =
['1']
A_non_active =
['-1', '3']
['2', '1']
b_non_active =
['8', '4']
dk =
['1', '-1']
-----LINE SEARCH-----
RHS =
['11', '5']
LHS =
['-4', '1']
a_max = 5
bounds =
['0', '5']
alpha = 5
x =
['5', '-6']
______________________________
ITERATION: 3
-----DIRECTION SEARCH-----
grad =
['4', '17']
active_index =
['2', '3']
A_active =
['2', '1']
['-1', '-1']
b_active =
['4', '1']
A_non_active =
['-1', '3']
b_non_active =
['8']
dk =
['0', '0']
Optimization terminated due to grad * dk = 0
